### Term 1 Project 3 : Behavioral Cloning
#### Project Writeup

##### Behavioral Cloning Project

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report
---
#### Rubric Points
Here I will consider the [rubric points](https://review.udacity.com/#!/rubrics/432/view) individually and describe how I addressed each point in my implementation

---
### Files Submitted & Code Quality

#### 1. Submission includes all required files and can be used to run the simulator in autonomous mode

My project includes the following files:

|File Name        |Purpose                                                                             |
|:----------------|------------------------------------------------------------------------------------|
|model.py         |Contains the script for the model, pre-processing and augmentation of data, training|
|drive.py         |Used for driving the car in autonomous mode using the model generated by model.py   | 
|model.h5         |Contains a trained convolution neural network                                       |
|video.mp4        |Autonomous run in the simulator for Track 1 based on the model generated by model.py|
|writeup_report.md|Contains project writeup                                                            |   

#### 2. Submission includes functional code
Using the Udacity provided simulator and my drive.py file, the car can be driven autonomously around the track by executing 
```sh
python drive.py model.h5
```

#### 3. Submission code is usable and readable

The model.py file contains the code for training and saving the convolution neural network. The file shows the pipeline I used for training and validating the model, and it contains comments to explain how the code works.

---
### Model Architecture and Training Strategy

#### 1. Model Architecture
My model is based on the NVIDIA End-End Deep Learning for Autonomous Cars.
I've modified the architecture to fit to my use case and tuned it for better results.

A pictorial representation of the model can be seen below

![Behavioral Cloning Model Architecture](ModelArchitecture.png)

I've built my model using Keras as it defines simple syntax for defining models so that I can concentrate more on pre-processing and augmenting the data.

Here's how my model is a good fit for this model

 **Input Layer** : It all starts with a 160 * 320 pixel Color Image fed to the network. The training data set is generated using the simulator training environment's video feed

 **Normalization Layer** : I've used Keras Lamdba layer to normalize the input. Normalizing and making the data zero-mean makes the model to train much faster as the gradient descent can be handled in a much better way for zero-mean numbers

 **Convolutional Layers** : Since we are dealing with images, the feautre classification is important to identify the corners and take decisions based on that. Convolutional layers help a lot for this problem.

 **Dropout Layers** : Overtraining the model could lead to over-fitting and this can be avoided by randomly dropping certain weights using dropout layers. This will help the model to train in such a way that it is not dependent on any set of parameters as the parameters can become zero anytime - thus decoupling the hyperparameters from the results

 **Fully Connected Layers** : The problem of identifying the steering angle is a regression problem and Fully Connected layers help in achieving the right angle based on the training model

 **Activations** : The model should be non-linear and in order to achieve that, ELU functions have been used for activating the neurons
 
 **Optimizer** : Adam (Adaptive Moment Estimation) Optimization is used during training. This is an extention to the Stochastic Gradient Descent method. In this method, instead of having a single learning rate during gradient descent, the algorithm computes individual adaptive learning rates for different parameters 

Here's a representation of the model with hyperparmeters in each layer

#### 2. Training & Testing
This essence of this project is to determine the steering angle based on the position of the center of the car with respect to its surroundings. This is a regression problem wherein the model should evaluate the best possible angle to feed in to the steering of the car. This model is derived based on training data created in a similar(need not be same) environment as test data

**Generating Training Data**
The training data is derived by driving the car in Udacity's Self Driving Car Simulator.

User is required to drive the car using keyboard controls trying to be in the center of the lane. This video feed is converted to pictures and the names of the pictures are associated steering angles at each frame.

The car has 3 cameras in the simulator, so each frame provides us with 3 different views of hte same image - a center image, an image from the left side of the windsheild, an image from the right side of the wind sheild.

The steering angle will always be the angle based on the center of the car. This angle can be manipulated to get the corresponding angles for left and right images

Note : Though the simulator lets you drive the car, I could not drive the car smoothly using my keyboard. I was unable to avoid crossing the side lanes multiple times. The model trained on the testing data I generated was not doing so great - as the training data itself was not proper. Hence I've decided to use the Udacity provided trained data. The results obtained with this training data was much better, and my car was always within the lane boundaries.

**Pre-Processing & Augmenting the Training Data**
Udacity's training data consisted of ~8500 images. These are the center, left & right images of each frame during the course of training set generation. Though the model can be trained based on this model, inorder to make the model more generic and adaptable to any environment, i've pre-processed and augmented the data

**Pre-processing**
- **Conversion of BGR to RGB** : This comes under pre-processing step. CV2 reads the image in BGR colorspace. But the drive.py which is used to test the model in the simulator reads the image in RGB colorspace. Hence the images are converted to RGB colorspace as soon as they are read

    Code Snippet `cv2.cvtColor(center_image_bgr, cv2.COLOR_BGR2RGB)`
    
- **Cropping the Image** : The image given by the camera in the simulator consists of the Car's dashboard at the bottom as well as the sky and trees at the top. These values do not really contribute anything for the model's decision. Hence to avoid considering these areas, the images are cropped. This is done in the Model itself rather than in the pre-processing pipeline, but to have a view on what's hapenning, here's the representation

    Code Snippet `model.add(Cropping2D(cropping=((70, 25), (0, 0)))) #70 is pixels from top and 25 is pixels from bottom`

- **Gaussian Blur** : This comes under pre-processing step. Inorder to avoid sudden spikes in the color ranges and to introduce normalization, Gaussian Blur is applied to the images

    Code Snippet `cv2.GaussianBlur(left_image, (3, 3), 0) #(3, 3) represents the kernel size while applying blur`

**Data Augmentation**
- **Image Translation** : To provide better training samples, each image will be translated laterally at a random factor, and the corresponding steering angle is calculated. This step will help the model to get more training samples with different angles to better train the model

    Code Snippet `cv2.warpAffine(X[i], trans_m, (width, height)) #trans_m is a matrix with the amount of translation required`
    
- **Image Flip** Note that the training data is based on a loop that is left aligned. If we train the model based on this data, the model will tend to move to the left mostly. Inorder to avoid this, for each image where the angle is greater than a particular value(which represents a curve), we flip the image and add to the training data. This leads to a generalized training set, and more number of training data

    Code Snippet `cv2.flip(image, 1) #1 represents flipping horizontally, and steering angles will be multiplied by -1`
    
- **Changing Brightness** The training data is also augmented by randomly changing the brightness of the image and adding to the training set. This gives us with more training data, with images representing different lighting conditions. This makes the model more generalized and can be adapted to different tracks and lighting conditions

    Code Snippet `hsv_img[:,:,2] =  hsv_img[:,:,2] * random_value #Image is converted to HSV channel where the last channel (V - Value) represents brightness`

Note : I've uesd Pre-processing steps and replaced the original images. Whereas during augmentation, i've added the transformed images to the existing training set which inturn increased the number of training samples



